In [3]:
import numpy
from sklearn.metrics import accuracy_score

from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, \
    TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, \
    KNeighborsTimeSeries
import pandas as pd
import numpy as np
import random
from tslearn.datasets import UCR_UEA_datasets
from tslearn.preprocessing import TimeSeriesResampler


In [101]:
X_test.shape

(100, 96, 1)

In [4]:
df = pd.read_pickle("Train.pkl")

In [15]:
randomlist1 = random.sample(range(0, 260), 78)

df_test1 = df.loc[df["Batch"].isin(randomlist1)]
df_train1 = df.loc[~df["Batch"].isin(randomlist1)]

In [16]:
df_steps1 = df_test1.loc[df_test1["CuStepNo ValueY"] != df_test1["Next_Step"]]
df_steps2 = df_train1.loc[df_train1["CuStepNo ValueY"] != df_train1["Next_Step"]]

In [17]:
y_train1 = df_steps2["DeviationID ValueY"].astype(int)
y_test1 = df_steps1["DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [11]:
78*4

312

In [25]:
y_train1

1052      4
1301      4
1372      4
1904      4
2102      5
         ..
264688    9
264886    9
265190    9
265261    9
265798    9
Name: DeviationID ValueY, Length: 692, dtype: int32

In [26]:
X_train1

,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,FIC14002_PV_Out ValueY,FIC14002_SP ValueY,YC14006_MV ValueY,...,LIC22002_SP ValueY,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY,Batch
1052,1,152.251556,100,0.075810,0.089699,100.000000,100.000000,0.000000,2000.0,12.5,...,2,3.475037,1.978950,15,10.0,0.116767,0.2,0,0.001709,1.0
1053,1,152.195923,100,0.074653,0.093750,100.000000,100.000000,0.000000,2000.0,12.5,...,2,9.951162,1.978154,15,10.0,0.116733,0.2,0,0.001718,1.0
1054,1,152.143127,100,0.075810,0.141204,100.000000,100.000000,0.000000,2000.0,12.5,...,2,14.227112,1.979745,15,10.0,0.116709,0.2,0,0.001707,1.0
1055,1,152.101624,100,0.080440,0.247685,100.000000,100.000000,0.000000,2000.0,12.5,...,2,20.453506,1.984520,15,10.0,0.116713,0.2,0,0.001692,1.0
1056,1,152.068481,100,0.078704,0.416667,99.328842,98.354942,796.162048,2000.0,12.5,...,2,24.727407,1.985315,15,10.0,0.116752,0.2,0,0.001694,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265993,3,98.288361,100,0.060185,0.078704,100.000000,100.000000,0.000000,2000.0,12.5,...,2,0.000000,2.016348,15,10.0,0.111238,0.2,0,0.001690,260.0
265994,3,98.364868,100,0.063657,0.078125,100.000000,100.000000,0.000000,2000.0,12.5,...,2,0.000000,2.011574,15,10.0,0.111272,0.2,0,0.001706,260.0
265995,3,98.364868,100,0.064236,0.080440,100.000000,100.000000,0.000000,2000.0,12.5,...,2,0.000000,2.012370,15,10.0,0.111278,0.2,0,0.001706,260.0
265996,3,98.434006,100,0.065394,0.079282,100.000000,100.000000,0.000000,2000.0,12.5,...,2,0.000000,2.001230,15,10.0,0.111320,0.2,0,0.001682,260.0


In [39]:
# X_train = []
# for i in X_train1.Batch.unique():
#     for j in X_train1["CuStepNo ValueY"].unique():
#         num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
#         alle = np.linspace(0, num.shape[0]-1, 100).round()
#         alle = [int(i) for i in alle]
#         num = [num[i] for i in alle]
#         num = np.atleast_3d(num)
#         num = num.reshape(1, 100, 40)
#         X_train.append(num)
    
X_train = []
for i in X_train1.Batch.unique():
    for j in X_train1["CuStepNo ValueY"].unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        # num = np.atleast_3d(num)
        # num = num.reshape(1, num.shape[0], 40)
        X_train.append(num)

In [42]:
X_train[0][0]

array([1.00000000e+00, 1.52251556e+02, 1.00000000e+02, 7.58101940e-02,
       8.96990299e-02, 1.00000000e+02, 1.00000000e+02, 0.00000000e+00,
       2.00000000e+03, 1.25000000e+01, 6.88657761e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.47503662e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.94473374e+00,
       2.00000000e+00, 0.00000000e+00, 1.98054111e+00, 2.00000000e+00,
       3.47503662e+00, 1.97894967e+00, 1.50000000e+01, 1.00000000e+01,
       1.16766751e-01, 2.00000003e-01, 0.00000000e+00, 1.70948007e-03])

In [108]:
X_train = np.array(X_train).reshape(700,100,40)

In [15]:
b = 200
for i in X_train1.Batch.unique():
    for j in X_train1["CuStepNo ValueY"].unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        num = np.atleast_3d(num)
        a = num.shape[0]
        if a < b:
            b = a
            print(num.shape)

(73, 41, 1)
(72, 41, 1)
(65, 41, 1)
(64, 41, 1)
(58, 41, 1)
(33, 41, 1)


In [22]:
num.shape

(1, 33, 41)

In [43]:
# X_test = []
# for i in X_test1.Batch.unique():
#     for j in X_test1["CuStepNo ValueY"].unique():
#         num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
#         alle = np.linspace(0, num.shape[0]-1, 100).round()
#         alle = [int(i) for i in alle]
#         num = [num[i] for i in alle]
#         num = np.atleast_3d(num)
#         num = num.reshape(1, 100, 40)
#         X_test.append(num)
X_test = []
for i in X_test1.Batch.unique():
    for j in X_test1["CuStepNo ValueY"].unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        # num = np.atleast_3d(num)
        # num = num.reshape(1, num.shape[0], 40)
        X_test.append(num)

In [46]:
from tslearn.utils import to_time_series_dataset

In [47]:
X = to_time_series_dataset(X_train)

In [54]:
X1 = to_time_series_dataset(X_test)

In [51]:
X[0].__len__()

927

In [113]:
X_test = np.array(X_test).reshape(304,100,40)

In [35]:
X_test.shape

(304, 100, 41)

In [44]:
X_train[0][0][0]

1.0

In [60]:
X_test1

,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,FIC14002_PV_Out ValueY,FIC14002_SP ValueY,YC14006_MV ValueY,...,LIC22002_SP ValueY,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY,Batch
0,1,149.985779,100,0.070602,0.087963,100.0,100.0,0.0,2000.0,12.5,...,2,0.611938,1.979745,15,10.0,0.127008,0.2,0,0.001710,0.0
1,1,149.985779,100,0.070602,0.086806,100.0,100.0,0.0,2000.0,12.5,...,2,1.151489,1.979745,15,10.0,0.127021,0.2,0,0.001715,0.0
2,1,150.027740,100,0.071759,0.103588,100.0,100.0,0.0,2000.0,12.5,...,2,11.073669,1.986111,15,10.0,0.127021,0.2,0,0.001712,0.0
3,1,150.043427,100,0.072338,0.173611,100.0,100.0,0.0,2000.0,12.5,...,2,11.819598,1.982133,15,10.0,0.126945,0.2,0,0.001698,0.0
4,1,150.015564,100,0.065972,0.326968,100.0,100.0,0.0,2000.0,12.5,...,2,22.069590,1.982133,15,10.0,0.126945,0.2,0,0.001700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263769,3,98.715607,100,0.064815,0.082176,100.0,100.0,0.0,2000.0,12.5,...,2,0.000000,2.020327,15,10.0,0.121545,0.2,0,0.001703,258.0
263770,3,98.790909,100,0.063079,0.083333,100.0,100.0,0.0,2000.0,12.5,...,2,0.000000,2.018735,15,10.0,0.121547,0.2,0,0.001711,258.0
263771,3,98.851746,100,0.064236,0.073495,100.0,100.0,0.0,2000.0,12.5,...,2,0.000000,2.010778,15,10.0,0.121547,0.2,0,0.001711,258.0
263772,3,98.898743,100,0.064236,0.077546,100.0,100.0,0.0,2000.0,12.5,...,2,0.000000,2.003617,15,10.0,0.121574,0.2,0,0.001713,258.0


In [59]:
X_train1

,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,FIC14002_PV_Out ValueY,FIC14002_SP ValueY,YC14006_MV ValueY,...,LIC22002_SP ValueY,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY,Batch
1052,1,152.251556,100,0.075810,0.089699,100.000000,100.000000,0.000000,2000.0,12.5,...,2,3.475037,1.978950,15,10.0,0.116767,0.2,0,0.001709,1.0
1053,1,152.195923,100,0.074653,0.093750,100.000000,100.000000,0.000000,2000.0,12.5,...,2,9.951162,1.978154,15,10.0,0.116733,0.2,0,0.001718,1.0
1054,1,152.143127,100,0.075810,0.141204,100.000000,100.000000,0.000000,2000.0,12.5,...,2,14.227112,1.979745,15,10.0,0.116709,0.2,0,0.001707,1.0
1055,1,152.101624,100,0.080440,0.247685,100.000000,100.000000,0.000000,2000.0,12.5,...,2,20.453506,1.984520,15,10.0,0.116713,0.2,0,0.001692,1.0
1056,1,152.068481,100,0.078704,0.416667,99.328842,98.354942,796.162048,2000.0,12.5,...,2,24.727407,1.985315,15,10.0,0.116752,0.2,0,0.001694,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265993,3,98.288361,100,0.060185,0.078704,100.000000,100.000000,0.000000,2000.0,12.5,...,2,0.000000,2.016348,15,10.0,0.111238,0.2,0,0.001690,260.0
265994,3,98.364868,100,0.063657,0.078125,100.000000,100.000000,0.000000,2000.0,12.5,...,2,0.000000,2.011574,15,10.0,0.111272,0.2,0,0.001706,260.0
265995,3,98.364868,100,0.064236,0.080440,100.000000,100.000000,0.000000,2000.0,12.5,...,2,0.000000,2.012370,15,10.0,0.111278,0.2,0,0.001706,260.0
265996,3,98.434006,100,0.065394,0.079282,100.000000,100.000000,0.000000,2000.0,12.5,...,2,0.000000,2.001230,15,10.0,0.111320,0.2,0,0.001682,260.0


In [62]:
df_train1

,Unnamed: 0,timestamp,DeviationID ValueY,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,...,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY,Next_Step,Batch
1052,1052,2018-10-11 10:21:45,4,1,152.251556,100,0.075810,0.089699,100.000000,100.000000,...,3.475037,1.978950,15,10.0,0.116767,0.2,0,0.001709,3.0,1.0
1053,1053,2018-10-11 10:21:46,4,1,152.195923,100,0.074653,0.093750,100.000000,100.000000,...,9.951162,1.978154,15,10.0,0.116733,0.2,0,0.001718,1.0,1.0
1054,1054,2018-10-11 10:21:47,4,1,152.143127,100,0.075810,0.141204,100.000000,100.000000,...,14.227112,1.979745,15,10.0,0.116709,0.2,0,0.001707,1.0,1.0
1055,1055,2018-10-11 10:21:48,4,1,152.101624,100,0.080440,0.247685,100.000000,100.000000,...,20.453506,1.984520,15,10.0,0.116713,0.2,0,0.001692,1.0,1.0
1056,1056,2018-10-11 10:21:49,4,1,152.068481,100,0.078704,0.416667,99.328842,98.354942,...,24.727407,1.985315,15,10.0,0.116752,0.2,0,0.001694,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265993,276643,2018-10-31 14:29:26,9,3,98.288361,100,0.060185,0.078704,100.000000,100.000000,...,0.000000,2.016348,15,10.0,0.111238,0.2,0,0.001690,3.0,260.0
265994,276644,2018-10-31 14:29:27,9,3,98.364868,100,0.063657,0.078125,100.000000,100.000000,...,0.000000,2.011574,15,10.0,0.111272,0.2,0,0.001706,3.0,260.0
265995,276645,2018-10-31 14:29:28,9,3,98.364868,100,0.064236,0.080440,100.000000,100.000000,...,0.000000,2.012370,15,10.0,0.111278,0.2,0,0.001706,3.0,260.0
265996,276646,2018-10-31 14:29:29,9,3,98.434006,100,0.065394,0.079282,100.000000,100.000000,...,0.000000,2.001230,15,10.0,0.111320,0.2,0,0.001682,3.0,260.0


In [130]:
#write me a proximity forest with dtw distance
from scipy.stats import mode

In [171]:
#dtw between the tow prior notebooks cells  
from tslearn.metrics import dtw
from tslearn.metrics import dtw_path
from tslearn.metrics import dtw_subsequence_path
from tslearn.metrics import dtw_path_from_metric
y_predicted = []
for k in X_test1.Batch.unique():
    for l in X_test1["CuStepNo ValueY"].unique():
        score = []
        for i in range(1,11):
            meanscore = []
            for j in df_train1[(df_train1["CuStepNo ValueY"] == 8)&(df_train1["DeviationID ValueY"] == i)].Batch.unique():
                meanscore.append(dtw(X_test1[(X_test1["CuStepNo ValueY"]==l)&(X_test1["Batch"]==k)].drop(["Batch"],axis=1),df_train1[(df_train1["DeviationID ValueY"]==i)&(df_train1["CuStepNo ValueY"]==l)&(df_train1["Batch"]==j)].drop(["DeviationID ValueY","Batch","Next_Step","timestamp","Unnamed: 0"],axis=1)))
            meanscore.sort()
            score.append(list(map(lambda x: (x,i), meanscore[:5])))
        flat_list = [item for sublist in score for item in sublist]
        flat_list.sort()
        y_predicted.append(mode([l[1] for l in flat_list[:5]])[0])

In [180]:
#dtw between the tow prior notebooks cells  
from tslearn.metrics import dtw
from tslearn.metrics import dtw_path
from tslearn.metrics import dtw_subsequence_path
from tslearn.metrics import dtw_path_from_metric
y_predicted = []
for k in X_test1.Batch.unique():
    for l in X_test1["CuStepNo ValueY"].unique():
        score = []
        for i in range(1,11):
            meanscore = []
            for j in df_train1[(df_train1["CuStepNo ValueY"] == 8)&(df_train1["DeviationID ValueY"] == i)].Batch.unique():
                meanscore.append(dtw(X_test1[(X_test1["CuStepNo ValueY"]==l)&(X_test1["Batch"]==k)].drop(["Batch"],axis=1),df_train1[(df_train1["DeviationID ValueY"]==i)&(df_train1["CuStepNo ValueY"]==l)&(df_train1["Batch"]==j)].drop(["DeviationID ValueY","Batch","Next_Step","timestamp","Unnamed: 0"],axis=1)))
            score.append(np.min(meanscore))
        y_predicted.append(np.argmin(score)+1)

In [181]:
y_predicted.__len__()

312

In [169]:
y_test1.__len__()

312

In [173]:
accuracy_score(y_test1, y_predicted)

0.7660256410256411

In [182]:
accuracy_score(y_test1, y_predicted)

0.7692307692307693

In [52]:
knn_clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, metric="dtw")
knn_clf.fit(X, y_train1)

KNeighborsTimeSeriesClassifier(n_neighbors=3)

In [57]:
predicted_labels = knn_clf.predict(X_test)

C:\Users\paulh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:856: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order, dtype=dtype)


ValueError: Expected 2D array, got 1D array instead:
array=[array([[1.00000000e+00, 1.49985779e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.71031873e-03],
        [1.00000000e+00, 1.49985779e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.71468942e-03],
        [1.00000000e+00, 1.50027740e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.71223748e-03],
        ...,
        [1.00000000e+00, 1.36731201e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74658047e-03],
        [1.00000000e+00, 1.36644073e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74658047e-03],
        [1.00000000e+00, 1.36576385e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74411153e-03]])
 array([[7.00000000e+00, 1.36526093e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74385821e-03],
        [7.00000000e+00, 1.36503082e+02, 1.00000000e+02, ...,
         2.00000003e-01, 

In [115]:
predicted_labels = knn_clf.predict(X_train)
accuracy_score(y_train1, predicted_labels)

0.84

In [140]:
accuracy_score(y_test1, predicted_labels)



0.7236842105263158

In [141]:
from sklearn.metrics import classification_report
print(classification_report(y_test1, predicted_labels))

              precision    recall  f1-score   support

           1       0.67      0.75      0.71        40
           2       0.98      1.00      0.99        48
           3       0.62      1.00      0.76        24
           4       0.89      0.50      0.64        32
           5       0.69      0.64      0.67        28
           6       0.52      0.75      0.61        20
           7       0.73      0.79      0.76        24
           8       1.00      0.89      0.94        28
           9       0.57      0.75      0.65        28
          10       0.40      0.12      0.19        32

    accuracy                           0.72       304
   macro avg       0.71      0.72      0.69       304
weighted avg       0.73      0.72      0.70       304



In [131]:
df1 = pd.read_csv("Batch_118.csv")

In [132]:
df1

,Unnamed: 0,timestamp,DeviationID ValueY,CuStepNo ValueY,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,...,LIC22002_PV_Out ValueY,LIC22002_SP ValueY,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY
0,124285,2018-10-19 13:31:57,4,1,143.376923,100,0.075810,0.088542,100.0,100.0,...,1.991681,2,1.453430,1.982133,15,10.0,0.074548,0.2,0,0.001745
1,124286,2018-10-19 13:31:58,4,1,143.457947,100,0.075810,0.090856,100.0,100.0,...,1.988498,2,7.056091,1.978154,15,10.0,0.074529,0.2,0,0.001741
2,124287,2018-10-19 13:31:59,4,1,143.517303,100,0.073495,0.114005,100.0,100.0,...,1.993273,2,11.913584,1.988498,15,10.0,0.074565,0.2,0,0.001748
3,124288,2018-10-19 13:32:00,4,1,143.556213,100,0.072917,0.204282,100.0,100.0,...,1.987702,2,17.138062,1.991681,15,10.0,0.074646,0.2,0,0.001735
4,124289,2018-10-19 13:32:01,4,1,143.563751,100,0.075810,0.355324,100.0,100.0,...,1.984520,2,22.570557,1.990885,15,10.0,0.074652,0.2,0,0.001738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,125323,2018-10-19 13:49:15,4,3,142.892029,100,0.074653,0.090856,100.0,100.0,...,2.012370,2,0.000000,2.014757,15,10.0,0.065532,0.2,0,0.001732
1039,125324,2018-10-19 13:49:16,4,3,142.842712,100,0.075231,0.085069,100.0,100.0,...,2.004413,2,0.000000,2.023510,15,10.0,0.065555,0.2,0,0.001743
1040,125325,2018-10-19 13:49:17,4,3,142.819000,100,0.074653,0.084491,100.0,100.0,...,1.996455,2,0.000000,2.016348,15,10.0,0.065585,0.2,0,0.001740
1041,125326,2018-10-19 13:49:18,4,3,142.810425,100,0.075231,0.090856,100.0,100.0,...,1.997251,2,0.000000,2.003617,15,10.0,0.065580,0.2,0,0.001741


In [137]:
predicted_labels = knn_clf.predict(np.array(df1.loc[df1["CuStepNo ValueY"] == 1].drop(["DeviationID ValueY","timestamp","Unnamed: 0"],axis=1)).reshape(1,247,40))

In [138]:
predicted_labels

array([4])

In [136]:
#deviation id 3
np.array(df1.loc[df1["CuStepNo ValueY"] == 1].drop(["DeviationID ValueY","timestamp","Unnamed: 0"],axis=1)).shape

(247, 40)

In [17]:
numpy.random.seed(0)
n_ts_per_blob, sz, d, n_blobs = 20, 100, 1, 2

In [20]:
X, y = random_walk_blobs(n_ts_per_blob=n_ts_per_blob,
                         sz=sz,
                         d=d,
                         n_blobs=n_blobs)

In [160]:
X.shape

(40, 100, 1)

In [30]:
y.shape

(40,)

In [31]:
scaler = TimeSeriesScalerMinMax(value_range=(0., 1.))  # Rescale time series
X_scaled = scaler.fit_transform(X)

In [32]:
X_scaled

array([[[0.31135053],
        [0.3391987 ],
        [0.46099254],
        ...,
        [0.55517097],
        [0.51021691],
        [0.57177624]],

       [[0.29505959],
        [0.46729981],
        [0.42647398],
        ...,
        [0.60912615],
        [0.63228332],
        [0.62142942]],

       [[0.29817957],
        [0.29167574],
        [0.45061391],
        ...,
        [0.50771989],
        [0.48678425],
        [0.56133889]],

       ...,

       [[0.14845432],
        [0.0619924 ],
        [0.03227213],
        ...,
        [0.36138493],
        [0.58999307],
        [0.52613367]],

       [[0.2800383 ],
        [0.27368325],
        [0.11979288],
        ...,
        [0.3940308 ],
        [0.50554966],
        [0.5104046 ]],

       [[0.1213701 ],
        [0.13881785],
        [0.        ],
        ...,
        [0.38526045],
        [0.42681504],
        [0.58192767]]])

In [6]:
y.shape

(20,)

In [34]:
indices_shuffle = numpy.random.permutation(n_ts_per_blob * n_blobs)
X_shuffle = X_scaled[indices_shuffle]
y_shuffle = y[indices_shuffle]

In [35]:
X_shuffle

array([[[0.16487974],
        [0.0538851 ],
        [0.        ],
        ...,
        [0.39009502],
        [0.49068955],
        [0.47237332]],

       [[0.1213701 ],
        [0.13881785],
        [0.        ],
        ...,
        [0.38526045],
        [0.42681504],
        [0.58192767]],

       [[0.31135053],
        [0.3391987 ],
        [0.46099254],
        ...,
        [0.55517097],
        [0.51021691],
        [0.57177624]],

       ...,

       [[0.22847164],
        [0.15624171],
        [0.        ],
        ...,
        [0.35720629],
        [0.52029148],
        [0.63263464]],

       [[0.31319268],
        [0.30327943],
        [0.31001215],
        ...,
        [0.50147379],
        [0.50117598],
        [0.55552663]],

       [[0.35073234],
        [0.38923691],
        [0.43964006],
        ...,
        [0.62025088],
        [0.48576343],
        [0.58878751]]])

In [13]:
from tslearn.svm import TimeSeriesSVC

In [14]:
clf = TimeSeriesSVC(kernel="gak", gamma="auto", probability=True)
clf.fit(X, y).predict(X).shape
(20,)
sv = clf.support_vectors_
len(sv)  # should be equal to the nr of classes in the clf problem
2
sv[0].shape  
(..., 64, 2)
sv_sum = sum([sv_i.shape[0] for sv_i in sv])
sv_sum == clf.svm_estimator_.n_support_.sum()
True
clf.decision_function(X).shape
(20,)
clf.predict_log_proba(X).shape
(20, 2)
clf.predict_proba(X).shape
(20, 2)

(20, 2)